In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression

Создадим датасэт с ценой консультации доктора в медицинском центре. 
* Укажем рост врача, стаж и число сертификатов, а также цену за консультацию, которая зависит от категории врача. 
* Категория зависит от стажа (1 категория за каждые три года стажа) и сертификатов (две звезды за сертификат). 
* Есть до 7 сертификатов, доктора работают до 26 лет в мед центре после чего уходят на заслуженную высокооплачиваемую пенсию. 
* Каждая категория добавляет 1000 у.е. стоимости консультации. Признака "категория" пока не будет

In [34]:
n_samples = 1000

height = np.random.randint(150, 200, n_samples)
experience = np.random.randint(3, 26, n_samples)
certificates = np.random.randint(0, 4, n_samples)

price = (experience // 3 + certificates * 2) * 1000

df = pd.DataFrame({'height': height, 'experience': experience, 'certificates': certificates, 'price': price})
df.head(5)

,height,experience,certificates,price
0,175,10,1,5000
1,186,20,2,10000
2,192,3,1,3000
3,161,21,2,11000
4,192,21,3,13000


Делаем простую модель, которая учитывает все имеющиеся факторы и считаем ошибку.

In [35]:
from sklearn.metrics import mean_absolute_error

X = df[['height', 'experience', 'certificates']]
y = df['price']
model = LinearRegression().fit(X, y)
print('Weights: {}'.format(model.coef_))
print('Bias: {}'.format(model.intercept_))

predicted = model.predict(df[['height', 'experience', 'certificates']])
print('Error: {}'.format(mean_absolute_error(predicted, y)))

Weights: [-2.43391332e-01  3.32860752e+02  2.00231278e+03]
Bias: -282.2197747163682
Error: 222.5205101427503


In [36]:
median = df['price'].median()
mean_absolute_error(predicted, y) / median

0.031788644306107186

Ошибка не критическая, но все-таки есть - 3% от медианы. Уберем рост врача

In [38]:
X = df[['experience', 'certificates']]
y = df['price']
model = LinearRegression().fit(X, y)
print('Weights: {}'.format(model.coef_))
print('Bias: {}'.format(model.intercept_))

predicted = model.predict(df[['experience', 'certificates']])
print('Error: {}'.format(mean_absolute_error(predicted, y)))

Weights: [ 332.87007524 2002.16843843]
Bias: -324.6984821059341
Error: 222.35925926694384


Ошибка почти не изменилась. Добавляем признак "категория"

In [39]:
df['category'] = df['experience'] // 3 + df['certificates'] * 2
df.head(5)

,height,experience,certificates,price,category
0,175,10,1,5000,5
1,186,20,2,10000,10
2,192,3,1,3000,3
3,161,21,2,11000,11
4,192,21,3,13000,13


Используем только признак "категория"

In [41]:
X = df[['category']]
y = df['price']
model = LinearRegression().fit(X, y)
print('Weights: {}'.format(model.coef_))
print('Bias: {}'.format(model.intercept_))

predicted = model.predict(df[['category']])
print('Error: {}'.format(mean_absolute_error(predicted, y)))

Weights: [1000.]
Bias: -1.8189894035458565e-12
Error: 5.259153113001958e-13


То, что нужно.
Пример получился слишком искуственным и изначально модель возвращала очень низкую ошибку, вероятно из-за низкого размаха показателей. Но все равно напрашивается важный вывод - мы можем понимать как какие-то признаки опосредованно влияют на результат, но модели нужно дать что-то более конкретное. Длина и ширина квартиры могут быть разными с равной площадью, опыт и сертификация врачей может быть разной при равной категории. Важно немного подумать за модель, чтобы она работала лучше.